https://www.kaggle.com/code/manovirat/timeseries-using-prophet-hyperparameter-tuning

In [1]:
import pandas as pd
from prophet import Prophet
from prophet.diagnostics import cross_validation
from sklearn.model_selection import ParameterGrid
from prophet.diagnostics import performance_metrics
import random
import numpy as np
from sklearn.model_selection import TimeSeriesSplit

In [2]:
params_grid = {'seasonality_mode':('multiplicative','additive'),
               # set changepoint_prior_scale to 0.5 with steps of 0.005
               'changepoint_prior_scale':[x for x in np.arange(0.001, 0.5, 0.1)],
               'seasonality_prior_scale':[x for x in np.arange(0.01, 10, 2.5)],
              'holidays_prior_scale':[x for x in np.arange(0.01, 10, 2.5)]}

grid = ParameterGrid(params_grid)
cnt = 0
for p in grid:
    cnt = cnt+1

print('Total Possible Models',cnt)

Total Possible Models 160


In [3]:
#Define MASE Metric
def mean_absolute_scaled_error(y_true, y_pred):
    mase=0
    # Define numerator as the forecast error
    numerator = (np.abs(y_true - y_pred))

    # Define denominator as the mean absolute error of the in-sample one-step naive forecast
    y_true_ohne_1 = y_true[1:].reset_index(drop=True)
    y_true_ohne_ende = y_true[:-1].reset_index(drop=True)
    denominator = np.mean(np.abs(y_true_ohne_1 - y_true_ohne_ende))

    mase = np.mean(np.abs(numerator / denominator))

    return mase

In [4]:
# Load the data
data = pd.read_parquet('/Users/paddy/Documents/GitHub/Masterthesis_ML/data/03_15min_dataset.parquet')

# Convert the date column to datetime
data['date'] = pd.to_datetime(data['date'],format='%d/%m/%y %H:%M:%S').dt.strftime('%Y-%m-%d %H:%M:%S') 


In [5]:
# Feature Engineering

# Add a column with the time
data['time'] = [x for x in range(0, len(data))]

# Create the lagged features 24 +1
lags = 96 + 1

# Create the lagged features
for i in range(1, lags):
    data['lag' + str(i)] = data['count'].shift(i)

# Drop the first three rows
data = data.dropna().reset_index(drop=True)

# Rename column count to y
data = data.rename(columns={'count': 'y'})
data = data.rename(columns={'date': 'ds'})


train_data = data

In [6]:
cv_folds = TimeSeriesSplit(n_splits=5)

#convert data to only date not time
cv_data = train_data.copy()

#convert to timestamp with date format
#cv_data.ds = pd.to_datetime(cv_data.ds).dt.date

print(cv_data.ds)   

#Define cutoff dates for cross validation at end of train_data/cv_folds
cutoffs = []
# for train_index, test_index in cv_folds.split(train_data):
#     cutoffs.append(train_data.ds.iloc[test_index].max())
#     #convert to timestamp
#     cutoffs = pd.to_datetime(cutoffs)
    
# print(cutoffs)

for i, (train_index, test_index) in enumerate(cv_folds.split(cv_data)):
    print(f"Fold {i}:")
    print(f"  Train: index={train_index}")
    print(f"  Test:  index={test_index}")
    #get the date for the last train index
    cutoffs.append(cv_data.ds.iloc[train_index].max())
    #only date not time
    #cutoffs = pd.to_datetime(cutoffs).date

print(cutoffs)

# Define cutoffs for cross validation with 5 folds every two months with first cut on 2022-06-01, last day 2023-01-15 and horizon of 15 days
#cutoffs = pd.to_datetime(['2022-06-01', '2022-08-01', '2022-10-01', '2022-12-01', '2023-01-15'])
# print(cutoffs)


0        2022-04-02 00:00:00
1        2022-04-02 00:15:00
2        2022-04-02 00:30:00
3        2022-04-02 00:45:00
4        2022-04-02 01:00:00
                ...         
34939    2023-03-31 22:45:00
34940    2023-03-31 23:00:00
34941    2023-03-31 23:15:00
34942    2023-03-31 23:30:00
34943    2023-03-31 23:45:00
Name: ds, Length: 34944, dtype: object
Fold 0:
  Train: index=[   0    1    2 ... 5821 5822 5823]
  Test:  index=[ 5824  5825  5826 ... 11645 11646 11647]
Fold 1:
  Train: index=[    0     1     2 ... 11645 11646 11647]
  Test:  index=[11648 11649 11650 ... 17469 17470 17471]
Fold 2:
  Train: index=[    0     1     2 ... 17469 17470 17471]
  Test:  index=[17472 17473 17474 ... 23293 23294 23295]
Fold 3:
  Train: index=[    0     1     2 ... 23293 23294 23295]
  Test:  index=[23296 23297 23298 ... 29117 29118 29119]
Fold 4:
  Train: index=[    0     1     2 ... 29117 29118 29119]
  Test:  index=[29120 29121 29122 ... 34941 34942 34943]
['2022-06-01 15:45:00', '2022-08-01 07

In [7]:
MASE_list = []
best_mase = 100000000
best_params = None
for p in grid:
    train = pd.DataFrame(train_data)
    test = pd.DataFrame()
    print(p)
    random.seed(0)
    train_model =Prophet(changepoint_prior_scale = p['changepoint_prior_scale'],
                        holidays_prior_scale = p['holidays_prior_scale'],
                        seasonality_prior_scale = p['seasonality_prior_scale'],
                        seasonality_mode = p['seasonality_mode'],
                        weekly_seasonality=True,
                        daily_seasonality = True,
                        yearly_seasonality = False,
                        )
    
     # add regressors with lags 1 to 24
    for i in range(1,97):
        train_model.add_regressor('lag'+str(i), standardize=False)
        train['lag'+str(i)] = train['y'].shift(i)


    train_model.fit(train_data)
    
    df_cv = cross_validation(train_model, period='60 days', horizon='30 days', parallel="processes")

     # X and y to pandas dataframe
    X_train = pd.DataFrame(df_cv)

    y_true = df_cv['y'].values
    #make y_true as dataframe
    y_true = pd.DataFrame(y_true)

    y_pred = df_cv['yhat'].values
    #make y_pred as dataframe
    y_pred = pd.DataFrame(y_pred)

    MASE_list.append(mean_absolute_scaled_error(y_true, y_pred))
    


    # print('RMSE:',np.sqrt(MSE))
    # RMSE_list.append(np.sqrt(MSE))

# Find the best parameters
tuning_results = pd.DataFrame(grid)
tuning_results['mase'] = MASE_list
print(tuning_results)

{'changepoint_prior_scale': 0.001, 'holidays_prior_scale': 0.01, 'seasonality_mode': 'multiplicative', 'seasonality_prior_scale': 0.01}


08:08:38 - cmdstanpy - INFO - Chain [1] start processing
08:08:43 - cmdstanpy - INFO - Chain [1] done processing
08:08:46 - cmdstanpy - INFO - Chain [1] start processing
08:08:47 - cmdstanpy - INFO - Chain [1] done processing
08:08:47 - cmdstanpy - INFO - Chain [1] start processing
08:08:48 - cmdstanpy - INFO - Chain [1] start processing
08:08:49 - cmdstanpy - INFO - Chain [1] done processing
08:08:49 - cmdstanpy - INFO - Chain [1] start processing
08:08:50 - cmdstanpy - INFO - Chain [1] start processing
08:08:52 - cmdstanpy - INFO - Chain [1] done processing
08:08:56 - cmdstanpy - INFO - Chain [1] done processing
08:08:56 - cmdstanpy - INFO - Chain [1] done processing


{'changepoint_prior_scale': 0.001, 'holidays_prior_scale': 0.01, 'seasonality_mode': 'multiplicative', 'seasonality_prior_scale': 2.51}


08:09:03 - cmdstanpy - INFO - Chain [1] start processing
08:09:12 - cmdstanpy - INFO - Chain [1] done processing
08:09:14 - cmdstanpy - INFO - Chain [1] start processing
08:09:15 - cmdstanpy - INFO - Chain [1] start processing
08:09:15 - cmdstanpy - INFO - Chain [1] done processing
08:09:17 - cmdstanpy - INFO - Chain [1] start processing
08:09:18 - cmdstanpy - INFO - Chain [1] start processing
08:09:18 - cmdstanpy - INFO - Chain [1] done processing
08:09:18 - cmdstanpy - INFO - Chain [1] start processing
08:09:20 - cmdstanpy - INFO - Chain [1] done processing
08:09:28 - cmdstanpy - INFO - Chain [1] done processing
08:09:31 - cmdstanpy - INFO - Chain [1] done processing


{'changepoint_prior_scale': 0.001, 'holidays_prior_scale': 0.01, 'seasonality_mode': 'multiplicative', 'seasonality_prior_scale': 5.01}


...

{'changepoint_prior_scale': 0.401, 'holidays_prior_scale': 7.51, 'seasonality_mode': 'additive', 'seasonality_prior_scale': 5.01}


09:58:00 - cmdstanpy - INFO - Chain [1] start processing
09:58:12 - cmdstanpy - INFO - Chain [1] done processing
09:58:15 - cmdstanpy - INFO - Chain [1] start processing
09:58:16 - cmdstanpy - INFO - Chain [1] start processing
09:58:18 - cmdstanpy - INFO - Chain [1] start processing
09:58:19 - cmdstanpy - INFO - Chain [1] start processing
09:58:20 - cmdstanpy - INFO - Chain [1] start processing
09:58:20 - cmdstanpy - INFO - Chain [1] done processing
09:58:24 - cmdstanpy - INFO - Chain [1] done processing
09:58:30 - cmdstanpy - INFO - Chain [1] done processing
09:58:32 - cmdstanpy - INFO - Chain [1] done processing
09:58:35 - cmdstanpy - INFO - Chain [1] done processing


{'changepoint_prior_scale': 0.401, 'holidays_prior_scale': 7.51, 'seasonality_mode': 'additive', 'seasonality_prior_scale': 7.51}


09:58:39 - cmdstanpy - INFO - Chain [1] start processing
09:58:52 - cmdstanpy - INFO - Chain [1] done processing
09:58:55 - cmdstanpy - INFO - Chain [1] start processing
09:58:56 - cmdstanpy - INFO - Chain [1] start processing
09:58:57 - cmdstanpy - INFO - Chain [1] start processing
09:58:58 - cmdstanpy - INFO - Chain [1] start processing
09:58:59 - cmdstanpy - INFO - Chain [1] start processing
09:59:00 - cmdstanpy - INFO - Chain [1] done processing
09:59:03 - cmdstanpy - INFO - Chain [1] done processing
09:59:08 - cmdstanpy - INFO - Chain [1] done processing
09:59:10 - cmdstanpy - INFO - Chain [1] done processing
09:59:14 - cmdstanpy - INFO - Chain [1] done processing


     changepoint_prior_scale  holidays_prior_scale seasonality_mode  \
0                      0.001                  0.01   multiplicative   
1                      0.001                  0.01   multiplicative   
2                      0.001                  0.01   multiplicative   
3                      0.001                  0.01   multiplicative   
4                      0.001                  0.01         additive   
..                       ...                   ...              ...   
155                    0.401                  7.51   multiplicative   
156                    0.401                  7.51         additive   
157                    0.401                  7.51         additive   
158                    0.401                  7.51         additive   
159                    0.401                  7.51         additive   

     seasonality_prior_scale      mase  
0                       0.01  0.873724  
1                       2.51  0.862324  
2                       

In [8]:
best_params = grid[np.argmin(MASE_list)]
print('Best Parameters',best_params)
print('Best MASE',np.min(MASE_list))
print(MASE_list)

Best Parameters {'seasonality_prior_scale': 7.51, 'seasonality_mode': 'additive', 'holidays_prior_scale': 5.01, 'changepoint_prior_scale': 0.001}
Best MASE 0.8485734311780362
[0.8737241998031012, 0.8623244206194794, 0.8643600322198894, 0.8633942438826896, 0.8491082136024345, 0.8509281779015363, 0.8520324330000774, 0.8509039283599716, 0.8569513310256807, 0.8502703265214069, 0.8522163454115698, 0.85089462864976, 0.8509875935578669, 0.8501791386488574, 0.8499698080600231, 0.8515696120736731, 0.8573829431870728, 0.8498107918017389, 0.8505724287916001, 0.8505504016638085, 0.8532117442444216, 0.8515309640213918, 0.8512281661402635, 0.8485734311780362, 0.8580026527892654, 0.8491911825288488, 0.8511097334944123, 0.8505086295432451, 0.8518506843000079, 0.8499174898416486, 0.8510849989941993, 0.8510610853270383, 0.8855298867354822, 0.870162280095455, 0.8704133942431596, 0.8702128084590139, 0.8515863534395397, 0.8523605454404332, 0.852690263548527, 0.8520629669800855, 0.8659028167788736, 0.856238